<a href="https://colab.research.google.com/github/plaban1981/Timeseries/blob/master/TimeSeries_Prediction_of_Corona_Virus_outbreak_using_PROPHET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Coronavirus Outbreak and Recovering Rate with FB Prophet

* Lots of influencing factors hence prediction can be wrong
* Less Data
* Presence of Effective Treatment and Vaccine can influence prediction

#### Installation
pip install fbprophet

#### Data Source
https://github.com/CSSEGISandData/COVID-19

https://raw.githubusercontent.com/Jcharis/DataScienceTools/master/Data_analysis_of_Coronavirus_Outbreak_with_Python/coronavirus_data_clean.csv

https://github.com/Jcharis/DataScienceTools/blob/master/Data_analysis_of_Coronavirus_Outbreak_with_Python/Time_Series_Prediction_of_Coronavirus_Outbreak_with_Facebook_Prophet.ipynb


In [0]:

confirmed_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
recovered_cases_url ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
death_cases_url ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
df = pd.read_csv(confirmed_cases_url)
df[df['Country/Region'] == 'India']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20
131,NaN,India,21.0,78.0,0,0,0,0,0,0,0,0,1,1,1,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...,1024,1251,1397,1998,2543,2567,3082,3588,4778,5311,5916,6725,7598,8446,9205,10453,11487,12322,13430,14352,15722,17615,18539,20080,21370,23077,24530,26283,27890,29451,31324,33062,34863,37257,39699,42505,46437,49400,52987,56351


## Massage a DataFrame into a format where one or more columns are identifier

In [0]:
def get_n_melt_data(data_url,case_type):
    df = pd.read_csv(data_url)
    melted_df = df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'])
    melted_df.rename(columns={"variable":"Date","value":case_type},inplace=True)
    return melted_df

def merge_data(confirm_df,recovered_df,deaths_df):
	new_df = confirm_df.join(recovered_df['Recovered']).join(deaths_df['Deaths'])
	return new_df

In [0]:
confirm_df = get_n_melt_data(confirmed_cases_url,"Confirmed")
recovered_df = get_n_melt_data(recovered_cases_url,"Recovered")
deaths_df = get_n_melt_data(death_cases_url,"Deaths")

In [12]:
confirm_df[confirm_df['Country/Region'] == 'India'].tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
27263,NaN,India,21.0,78.0,5/3/20,42505
27529,NaN,India,21.0,78.0,5/4/20,46437
27795,NaN,India,21.0,78.0,5/5/20,49400
28061,NaN,India,21.0,78.0,5/6/20,52987
28327,NaN,India,21.0,78.0,5/7/20,56351


In [13]:
recovered_df [recovered_df ['Country/Region'] == 'India'].tail()

,Province/State,Country/Region,Lat,Long,Date,Recovered
25829,NaN,India,21.0,78.0,5/3/20,11775
26081,NaN,India,21.0,78.0,5/4/20,12847
26333,NaN,India,21.0,78.0,5/5/20,14142
26585,NaN,India,21.0,78.0,5/6/20,15331
26837,NaN,India,21.0,78.0,5/7/20,16776
